In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import preprocessing

import random

directory = 'data/DroneRF/RF_Data_FFT/'

In [4]:
!cd data/DroneRF/RF_Data_FFT && ls

00000_FFT_.csv	     10010_FFT_power.csv  10110_FFT_.csv
00000_FFT_power.csv  10011_FFT_.csv	  10110_FFT_power.csv
10000_FFT_.csv	     10011_FFT_power.csv  10111_FFT_.csv
10000_FFT_power.csv  10100_FFT_.csv	  10111_FFT_power.csv
10001_FFT_.csv	     10100_FFT_power.csv  11000_FFT_.csv
10001_FFT_power.csv  10101_FFT_.csv	  11000_FFT_power.csv
10010_FFT_.csv	     10101_FFT_power.csv


In [5]:
bepop = np.loadtxt(directory+'10000_FFT_power.csv', delimiter=",")
phantomDrone1 = np.loadtxt(directory+'11000_FFT_power.csv',delimiter=",")
backgroundRFnoise = np.loadtxt(directory+'00000_FFT_power.csv',delimiter = ",")
print(backgroundRFnoise.shape)
random.shuffle(backgroundRFnoise)
noDrones = backgroundRFnoise[:,:]


(2048, 4100)


In [6]:
phantomDrone1.shape

(2048, 2100)

In [7]:
bepop.shape

(2048, 2100)

In [11]:
bepop_transpose = np.transpose(bepop)
phantomdrone_transpose = np.transpose(phantomDrone1)
bepopDrone1_transpose = np.vstack((bepop_transpose,phantomdrone_transpose))
bepopDrone1_transpose = bepopDrone1_transpose[0:4100,:]


In [12]:
# Input Features is in rows when generated from matlab
# Transpose the 2D array to use the StandardScaler()
#bepopDrone1_transpose= np.transpose(bepopDrone1)
noDrones_transpose = np.transpose(noDrones)
bepopDrone1_transpose.shape
#noDrones_transpose.shape

(4100, 2048)

In [13]:
normalised_bepopDrone1 = preprocessing.StandardScaler().fit_transform(bepopDrone1_transpose)
normalised_noDrones = preprocessing.StandardScaler().fit_transform(noDrones_transpose)



In [14]:
enc = preprocessing.OneHotEncoder()
le = preprocessing.LabelEncoder()


In [15]:
type(normalised_bepopDrone1)

numpy.ndarray

In [16]:
Label_bepopDrone1 = np.array([1])
bepopDron1_dataset_concatenate = np.tile(Label_bepopDrone1[np.newaxis,:],(normalised_bepopDrone1.shape[0],1))
bepopDrone1_dataset=np.concatenate((normalised_bepopDrone1,bepopDron1_dataset_concatenate),axis=1)

Label_noDrones = np.array([0])
noDrones_dataset_concatenate = np.tile(Label_noDrones[np.newaxis,:],(normalised_noDrones.shape[0],1))
noDrones_dataset =np.concatenate((normalised_noDrones,noDrones_dataset_concatenate),axis =1 )

In [17]:
bepopDrone1_dataset

array([[-0.19144631, -0.23254951, -0.22885139, ..., -0.1942947 ,
        -0.18225159,  1.        ],
       [-0.19149772, -0.2325692 , -0.22896131, ..., -0.19428483,
        -0.1822094 ,  1.        ],
       [-0.1914971 , -0.23254434, -0.22896651, ..., -0.19427131,
        -0.18232803,  1.        ],
       ...,
       [-0.19150202, -0.23256628, -0.22899733, ..., -0.19415323,
        -0.18228557,  1.        ],
       [-0.19148834, -0.23257686, -0.22897307, ..., -0.1942747 ,
        -0.18231359,  1.        ],
       [-0.19148604, -0.23256055, -0.22897486, ..., -0.19428962,
        -0.18232195,  1.        ]])

In [18]:
noDrones_dataset.shape

(4100, 2049)

In [19]:
bepopDrone1_dataset.shape

(4100, 2049)

In [20]:
noDrones_dataset[2099]

array([ 1.66354217, -0.34405672, -0.34405672, ..., -0.13841169,
       -0.08861693,  0.        ])

In [21]:
stacked_dataset = np.vstack((bepopDrone1_dataset, noDrones_dataset))

In [22]:
stacked_dataset

array([[-0.19144631, -0.23254951, -0.22885139, ..., -0.1942947 ,
        -0.18225159,  1.        ],
       [-0.19149772, -0.2325692 , -0.22896131, ..., -0.19428483,
        -0.1822094 ,  1.        ],
       [-0.1914971 , -0.23254434, -0.22896651, ..., -0.19427131,
        -0.18232803,  1.        ],
       ...,
       [ 0.57683149, -0.75667933, -0.75667933, ..., -0.18543922,
        -0.10232871,  0.        ],
       [-0.74436664, -0.93065783, -0.93065783, ..., -0.35228169,
        -0.04798456,  0.        ],
       [ 1.55650696, -0.85797956, -0.85797956, ...,  1.41496731,
        -0.09445494,  0.        ]])

In [55]:
np.random.shuffle(stacked_dataset)

In [56]:
stacked_dataset

array([[-0.74923291,  1.78100912,  1.78100912, ..., -0.47189365,
        -0.0924968 ,  0.        ],
       [-0.19149918, -0.23252786, -0.22894828, ..., -0.19427812,
        -0.18232983,  1.        ],
       [-0.84354728, -0.77577511, -0.77577511, ...,  0.7751432 ,
        -0.09612119,  0.        ],
       ...,
       [-0.20563393, -0.62204808, -0.62204808, ..., -0.12320023,
        -0.11006101,  0.        ],
       [ 6.93155697,  2.98118683,  2.9414281 , ..., -0.08179136,
         0.64772588,  1.        ],
       [-0.19148386, -0.23256826, -0.22895362, ..., -0.19426367,
        -0.18230526,  1.        ]])

In [57]:
#X_train, X_test, y_train, y_test = train_test_split(
    #X, y, test_size=0.33, random_state=42)

In [58]:
TrainingSet_index = int(0.7*stacked_dataset.shape[0]) #2940
Validation_index = int(0.2*stacked_dataset.shape[0])#840
Test_index = int(stacked_dataset.shape[0] - (TrainingSet_index + Validation_index)) #420
stacked_dataset.shape

(8200, 2049)

In [59]:
Training_set = stacked_dataset[0:TrainingSet_index]
Validation_set = stacked_dataset[TrainingSet_index:Validation_index+TrainingSet_index]
#Test_set = stacked_dataset[Validation_index+TrainingSet_index:4200]
Test_set = stacked_dataset[Validation_index+TrainingSet_index:8200]



In [60]:
#X_Trained = Training_set[:,0:2048]
#Y_Trained = Training_set[:,2048:2049]

#X_Validation = Validation_set[:,0:2048]
#Y_Validation = Validation_set[:,2048:2049]

#X_Test = Test_set[:,0:2048]
#Y_TrueClassification = Test_set[:,2048:2049]

In [61]:
X_Trained = Training_set[:,0:2048]
Y_Trained = Training_set[:,2048:2049]

X_Validation = Validation_set[:,0:2048]
Y_Validation = Validation_set[:,2048:2049]

X_Test = Test_set[:,0:2048]
Y_TrueClassification = Test_set[:,2048:2049]

In [62]:
X_Test

array([[-1.91450150e-01, -2.32549965e-01, -2.28994263e-01, ...,
        -1.95881007e-01, -1.94290818e-01, -1.82309414e-01],
       [-7.80989885e-01, -5.43903340e-01, -5.43903340e-01, ...,
        -1.21025448e-01, -5.18091754e-01, -1.07948673e-01],
       [-7.27792303e-02,  4.89664274e-03, -1.30902857e-01, ...,
        -1.58228647e-01, -1.39900919e-01, -1.15961807e-01],
       ...,
       [-2.05633929e-01, -6.22048075e-01, -6.22048075e-01, ...,
         1.89242232e-03, -1.23200228e-01, -1.10061011e-01],
       [ 6.93155697e+00,  2.98118683e+00,  2.94142810e+00, ...,
         3.01674689e-01, -8.17913598e-02,  6.47725884e-01],
       [-1.91483857e-01, -2.32568256e-01, -2.28953625e-01, ...,
        -1.95838369e-01, -1.94263671e-01, -1.82305259e-01]])

In [63]:
X_Trained.shape

(5740, 2048)

In [71]:
#defining Keras model

model = Sequential()
model.add(Dense(2048, input_dim=2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
#model.add(Dense(512,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))


In [65]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [66]:
model.fit(X_Trained, Y_Trained, epochs=150, batch_size=10)
#model.fit(X_Trained, Y_Trained, epochs=150)

Epoch 1/150
574/574 [==============================] - 1s 2ms/step - loss: 1.0521 - accuracy: 0.9697
Epoch 2/150
574/574 [==============================] - 1s 2ms/step - loss: 3.0432e-04 - accuracy: 1.0000
Epoch 3/150
574/574 [==============================] - 1s 2ms/step - loss: 6.9363e-05 - accuracy: 1.0000
Epoch 4/150
574/574 [==============================] - 1s 2ms/step - loss: 2.0071e-05 - accuracy: 1.0000
Epoch 5/150
574/574 [==============================] - 1s 2ms/step - loss: 1.0236e-05 - accuracy: 1.0000
Epoch 6/150
574/574 [==============================] - 1s 2ms/step - loss: 6.2615e-06 - accuracy: 1.0000
Epoch 7/150
574/574 [==============================] - 1s 2ms/step - loss: 4.1038e-06 - accuracy: 1.0000
Epoch 8/150
574/574 [==============================] - 1s 2ms/step - loss: 2.7818e-06 - accuracy: 1.0000
Epoch 9/150
574/574 [==============================] - 1s 2ms/step - loss: 1.9533e-06 - accuracy: 1.0000
Epoch 10/150
574/574 [==============================] - 1s 

574/574 [==============================] - 1s 2ms/step - loss: 1.1545e-09 - accuracy: 1.0000
Epoch 79/150
574/574 [==============================] - 1s 2ms/step - loss: 2.9471e-10 - accuracy: 1.0000
Epoch 80/150
574/574 [==============================] - 1s 2ms/step - loss: 2.7239e-10 - accuracy: 1.0000
Epoch 81/150
574/574 [==============================] - 1s 2ms/step - loss: 4.0412e-10 - accuracy: 1.0000
Epoch 82/150
574/574 [==============================] - 1s 2ms/step - loss: 5.5440e-10 - accuracy: 1.0000
Epoch 83/150
574/574 [==============================] - 1s 2ms/step - loss: 7.1954e-10 - accuracy: 1.0000
Epoch 84/150
574/574 [==============================] - 1s 2ms/step - loss: 9.0104e-10 - accuracy: 1.0000
Epoch 85/150
574/574 [==============================] - 1s 2ms/step - loss: 1.0770e-09 - accuracy: 1.0000
Epoch 86/150
574/574 [==============================] - 1s 2ms/step - loss: 1.2924e-09 - accuracy: 1.0000
Epoch 87/150
574/574 [==============================] - 1s 

In [67]:
_, accuracy = model.evaluate(X_Validation, Y_Validation)
print('Accuracy: %.2f' % (accuracy*100))

52/52 [==============================] - 0s 1ms/step - loss: 0.0186 - accuracy: 0.9994
Accuracy: 99.94


In [68]:
model.predict(X_Test)

array([[1.00000000e+00],
       [6.33362178e-20],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.71414803e-14],
       [1.00000000e+00],
       [2.21343989e-23],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [2.08314250e-18],
       [2.04238638e-15],
       [1.00000000e+00],
       [1.00000000e+00],
       [2.93641474e-17],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.91462074e-14],
       [3.06011625e-32],
       [1.00000000e+00],
       [8.05024866e-11],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],


In [69]:
Y_TrueClassification

array([[1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

QN to ponder : With the trained model, using another drone characteristics input features, can we still classify Drone vs No Drones?

In [38]:
testing_Ardone = np.loadtxt(directory+'10100_FFT_power.csv', delimiter=",")

In [41]:
testing_Ardone= np.transpose(testing_Ardone)

In [44]:
normalised_Ardrone = preprocessing.StandardScaler().fit_transform(testing_Ardone)

In [53]:
model.predict(normalised_Ardrone[100:300,0:2048])

array([[0.9999995 ],
       [0.9999989 ],
       [0.9999989 ],
       [0.9999993 ],
       [0.9999994 ],
       [0.99999917],
       [0.9999994 ],
       [0.9999995 ],
       [0.9999995 ],
       [0.9999988 ],
       [0.9999993 ],
       [0.9999993 ],
       [0.9999994 ],
       [0.9999993 ],
       [0.9999995 ],
       [0.9999994 ],
       [0.9999993 ],
       [0.99999917],
       [0.99999964],
       [0.99999917],
       [0.99999917],
       [0.9999995 ],
       [0.9999995 ],
       [0.99999917],
       [0.9999994 ],
       [0.9999987 ],
       [0.9999989 ],
       [0.9999994 ],
       [0.9999995 ],
       [0.9999987 ],
       [0.99999917],
       [0.9999993 ],
       [0.99999857],
       [0.9999994 ],
       [0.9999993 ],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.9999987 ],
       [0.99999917],
       [0.9999995 ],
       [0.9999993 ],
       [0.99999905],
       [0.9999994 ],
       [0.9999987 ],
       [0.9999987 ],
       [0.9999989 ],
       [0.999

Answer: Yes, most small segments are classified as Drones[ Integer = 1]